In [ ]:
!pip install langchain langchain-core langchain-community pydantic duckduckgo_search ddgs langchain_experimental langchain-google-genai

In [ ]:

from langchain_core.tools import tool
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage
from dotenv import load_dotenv
import requests

load_dotenv()

In [ ]:
@tool 
def multiply(num1: int , num2: int) -> int:
    """Multiplies two numbers and returns the result."""
    return num1 * num2

In [ ]:
llm = ChatGoogleGenerativeAI(
    model="models/gemini-flash-latest",  
    temperature=0
)

In [ ]:
import os
print(os.getcwd())

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv(dotenv_path=".env", override=True)
EXCHANGERATE_API_KEY = os.getenv("EXCHANGERATE_API_KEY")

In [ ]:
llm.invoke("Hello, how are you today?")

In [ ]:
print(multiply.invoke({"num1": 5, "num2": 6}))

In [ ]:
multiply.name

In [ ]:
multiply.description

In [ ]:
multiply.args

# tool Binding 

In [ ]:
llm_with_tools = llm.bind_tools([multiply])

In [ ]:
llm_with_tools.invoke("Hi how are you")

In [ ]:
query= HumanMessage("What is the product of 7 and 8?")

In [ ]:
messages = [query]


In [ ]:
messages

In [ ]:
result =llm_with_tools.invoke(messages)

In [ ]:
messages.append(result)

In [ ]:
messages

In [ ]:
result.tool_calls

In [ ]:
multiply.invoke(result.tool_calls[0]['args'])

In [ ]:
tool_result = multiply.invoke(result.tool_calls[0])

In [ ]:
messages.append(tool_result)

In [ ]:
messages

In [ ]:
llm_with_tools.invoke(messages).content

# tool create

In [ ]:
import langchain_core
print(langchain_core.__version__)

In [ ]:
# tool create
from langchain_core.tools import InjectedToolArg
from typing import Annotated

@tool
def get_conversion_factor(base_currency: str, target_currency: str) -> float:
  """
  This function fetches the currency conversion factor between a given base currency and a target currency
  """
  url = f'https://v6.exchangerate-api.com/v6/c754eab14ffab33112e380ca/pair/{base_currency}/{target_currency}'

  response = requests.get(url)

  return response.json()

@tool
def convert(base_currency_value: int, conversion_rate: Annotated[float, InjectedToolArg]) -> float:
  """
  given a currency conversion rate this function calculates the target currency value from a given base currency value
  """

  return base_currency_value * conversion_rate


In [ ]:
get_conversion_fator.invoke({"base_currency":"USD", "target_currency":"INR"})
convert.invoke({"base_currency_value":100, "conversion_factor":82.5})

In [ ]:
# tool binding
llm_with_tools= llm.bind_tools([get_conversion_fator, convert])

In [ ]:
messages = [HumanMessage(" convert 100 USD to INR")]

In [ ]:
ai_message = llm_with_tools.invoke(messages)

In [ ]:
ai_message.tool_calls

In [ ]:
import json

for tool_call in ai_message.tool_calls:
  # execute the 1st tool and get the value of conversion rate
  if tool_call['name'] == 'get_conversion_factor':
    tool_message1 = get_conversion_factor.invoke(tool_call)
    # fetch this conversion rate
    conversion_rate = json.loads(tool_message1.content)['conversion_rate']
    # append this tool message to messages list
    messages.append(tool_message1)
  # execute the 2nd tool using the conversion rate from tool 1
  if tool_call['name'] == 'convert':
    # fetch the current arg
    tool_call['args']['conversion_rate'] = conversion_rate
    tool_message2 = convert.invoke(tool_call)
    messages.append(tool_message2)

